### Padam Jung Thapa

# Spring 2024: CSCI 6521 - Advanced Machine Learning I - PA 3

 Unfortunately, the training for the STATEFUL RNN was runned only upto 18 epochs as the runtime got disconnected, and the compute units were out as it was runned it colab. But the epochs for the STATELESS is runned for whole 50 epochs. If there would have been even a more day, then I would have submitted the full epochs for the StatelFUL RNN running it through the server. Sorry for the uncomplete epochs again, so I just picked the saved model of 18 epochs and ran it.

## 1. Load the text data from the Sentiment_Data folder and combine the files as a single text and call this as combined text sentiment_text.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [3]:
%cd drive/MyDrive/PA3_AML-I/

/content/drive/MyDrive/PA3_AML-I


In [4]:
#!ls

In [5]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tensorflow-addons
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
# assert tf.__version__ >= "2.`0"

#if not tf.test.is_gpu_available():
if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

import os
import numpy as np
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 7.8 MB/s eta 0:00:00
No GPU was detected. LSTMs and CNNs can be very slow without a GPU.
Go to Runtime > Change runtime and select a GPU hardware accelerator.


In [7]:
import os
from os import walk
current_dir = os.getcwd() # First, get the path of the working directory
path1 =current_dir+'/Sentiment_data/negative'
path2 =current_dir+'/Sentiment_data/positive'
_, _, list1 = next(os.walk(path1))
_, _, list2 = next(os.walk(path2))
filename_with_path=current_dir+'/Sentiment_data/'+'Text_Data.txt'
f1= open(filename_with_path,"w+")
# f.close()
for i in range(len(list1)):
    f1= open(filename_with_path,"a+")
    data=""
    f = open(path1+'/'+list1[i])
    data=f.read()
    f.close()
    f = open(path2+'/'+list2[i])
    data=data+f.read()
    f.close()
    f1.write(data)
    f1.close()
#This code creates the dataset "Text_Data.txt" under ./data
path_to_file = current_dir+'/Text_Data.txt'

In [8]:
filepath = current_dir+'/Sentiment_data/Text_Data.txt'
with open(filepath) as f:
    sentiment_text = f.read()

In [9]:
print(sentiment_text[:500])

 " spawn " features good guys , bad guys , lots of fighting , bloody violence , a leather-clad machine gun chick , gooey , self-healing bullet holes , scatological humor and a man-eating monster . 
it not only appears to have been tailor made for a swarm of 12- and 13-year-old boys , it appears to have been made by them . 
in a classic example of telling and not showing , " spawn " opens with a truckload of mumbo jumbo about forces of darkness , forces of light and how " men are the ones who cre


In [10]:
# number of words in the dataset
"".join(sorted(set(sentiment_text.lower())))

'\x05\n\x12\x13\x14\x16 !"#$%&\'()*+,-./0123456789:;=>?@[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~'

In [11]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(sentiment_text)

In [12]:
tokenizer.texts_to_sequences(["Second"])

[[7, 2, 13, 6, 8, 12]]

In [13]:
tokenizer.sequences_to_texts([[21, 6, 9, 8]])

['b o r n']

In [14]:
max_id = len(tokenizer.word_index) # number of distinct characters => max_id = 39 in this case.
dataset_size = tokenizer.document_count # total number of characters
dataset_size

7786004

In [15]:
# print all the unique characters in the dataset
uniq_chars = "".join(sorted(set(sentiment_text.lower())))
uniq_chars, len(uniq_chars)

('\x05\n\x12\x13\x14\x16 !"#$%&\'()*+,-./0123456789:;=>?@[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~',
 74)

## 2. Build up a stateless character-RNN model for text generation, trained using sentiment_text and train the model for at least 50 epochs

In [ ]:
# check for GPU
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
np.random.seed(2635493)
tf.random.set_seed(2635493)

In [ ]:
# tokenizing the sentiments_text
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(sentiment_text)

In [ ]:
tokenizer.texts_to_sequences("Led Zeppelin")

[[11], [2], [12], [1], [34], [2], [19], [19], [2], [11], [5], [8]]

In [ ]:
tokenizer.sequences_to_texts([[11], [2], [12], [1], [34], [2], [19], [19], [2], [11], [5], [8]])

['l', 'e', 'd', ' ', 'z', 'e', 'p', 'p', 'e', 'l', 'i', 'n']

In [ ]:
dataset_size = tokenizer.document_count
dataset_size

7786004

In [ ]:
# get the number of unique characters
max_ids = len(tokenizer.word_index)
max_ids

74

In [ ]:
# Encode the full dataset
[encoded_data] = np.array(tokenizer.texts_to_sequences([sentiment_text])) - 1 # subtracted 1 so it starts from 0 rather than 1
[encoded_data]

[array([ 3,  6,  6, ..., 23,  0, 24])]

In [ ]:
# length of encoded data
len(encoded_data)

7786004

In [ ]:
train_data_size = dataset_size * 90 // 100 # 90 % of the total data
dataset = tf.data.Dataset.from_tensor_slices(encoded_data[:train_data_size])

In [ ]:
# chopping the dataset into multiple windows
n_steps = 100
window_length = n_steps + 1  # target is 1 character ahead of input
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

In [ ]:
# convert the nested dataset into flat data set
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [ ]:
# splitting dataset into batches and shuffle it
batch_size = 64
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [ ]:
# one hot encode the dataset
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_ids), Y_batch)
)

In [ ]:
# add prefetch to the dataset
dataset = dataset.prefetch(1)

In [ ]:
# print the shape of a batch
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(64, 100, 74) (64, 100)


### Modeling stateless RNN

While training a stateless RNN, it was observed that the loss function, specifically sparse categorical crossentropy, was returning NaN values, which posed issues during predictions. Consequently, I developed a callback function using keras.callbacks.Callback designed to save the model at the end of each epoch and stop the training process upon encountering NaN values. This approach also facilitates the reloading of the model from the last successful epoch, given the extensive training durations and the likelihood that the training may not reach the 50-epoch mark.

In [ ]:
# call back funciton
class SaveModelAndHaltTraining(keras.callbacks.Callback):
    def __init__(self, prefix):
        super().__init__()
        self.prefix = prefix

    def on_epoch_end(self, epoc, logs):

        epoc = epoc + 1
        # set file name to save data
        filename = self.prefix + "_ep_" + str(epoc) + ".h5" # save the model in HDF5 format

        # save model
        print(f"\nSaving file: {filename}")
        self.model.save(filename)

        loss = logs.get('loss')
        print(f"Epoch loss: {loss}")

        # halt training if loss is nan
        if np.isnan(loss):
            print(f"\nNaN detected at Epoch: {epoc}. Training Halted!!!")
            self.model.stop_training = True


In [ ]:
save_model_and_halt_training = SaveModelAndHaltTraining('stateless_rnn')

# Modeling the stateless RNN
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_ids]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_ids, activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, steps_per_epoch=train_data_size // batch_size,
                    epochs=50, callbacks=[save_model_and_halt_training])

Epoch 1/50
109489/109490 [============================>.] - ETA: 0s - loss: 1.1924
Saving file: stateless_rnn_ep_1.h5
Epoch loss: 1.1924208402633667
109490/109490 [==============================] - 1194s 11ms/step - loss: 1.1924
Epoch 2/50
109490/109490 [==============================] - ETA: 0s - loss: 1.2934
Saving file: stateless_rnn_ep_2.h5
Epoch loss: 1.2934210300445557
109490/109490 [==============================] - 1192s 11ms/step - loss: 1.2934
Epoch 3/50
109488/109490 [============================>.] - ETA: 0s - loss: 1.3884
Saving file: stateless_rnn_ep_3.h5
Epoch loss: 1.3884317874908447
109490/109490 [==============================] - 1206s 11ms/step - loss: 1.3884
Epoch 4/50
109490/109490 [==============================] - ETA: 0s - loss: 1.3980
Saving file: stateless_rnn_ep_4.h5
Epoch loss: 1.3979785442352295
109490/109490 [==============================] - 1214s 11ms/step - loss: 1.3980
Epoch 5/50
109489/109490 [============================>.] - ETA: 0s - loss: 1.4321
S

In [19]:
# define preprocessing function
def preprocess(text):
    '''
    Tokenizes the text and one hot encodes it.
    Parameter:
    - text: text to be processed (string)
    Returns:
    - tokenized and one hot encoded text
    '''

    X = np.array(tokenizer.texts_to_sequences(text)) - 1

    return tf.one_hot(X, max_ids)

In [ ]:
# load the saved model
sl_model_path = 'stateless_rnn_ep_50.h5'
loaded_sl_model = keras.models.load_model(sl_model_path)

In [ ]:
# test models prediction on custom text
X_new = preprocess(["Hi, how are yo"])
y_pred = loaded_sl_model(X_new)

In [ ]:
y_pred

<tf.Tensor: shape=(1, 14, 74), dtype=float32, numpy=
array([[[8.7085553e-02, 3.7312010e-01, 3.4331091e-02, ...,
         2.8405225e-29, 1.2320271e-29, 2.4005706e-29],
        [3.8505176e-03, 3.0409612e-02, 2.4892900e-02, ...,
         1.1522060e-36, 2.4371925e-36, 1.5556761e-36],
        [9.8825675e-01, 1.7205573e-03, 1.6501863e-04, ...,
         1.5698905e-29, 1.2562754e-29, 6.0515438e-30],
        ...,
        [1.2671677e-04, 2.6968997e-02, 8.7144718e-02, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [1.0112445e-02, 2.0506488e-01, 8.0967980e-04, ...,
         3.3335153e-37, 8.8215922e-37, 5.2585213e-36],
        [6.7294040e-04, 2.1548308e-03, 2.2797259e-03, ...,
         1.3809284e-30, 5.1714238e-31, 3.0775221e-30]]], dtype=float32)>

In [ ]:
y_pred_class = np.argmax(y_pred, axis=-1)
text_pred = tokenizer.sequences_to_texts(y_pred_class + 1)[0][-1]

In [ ]:
text_pred

'u'

## 3. Build up a stateful character-RNN model for text generation, trained using sentiment_text. Train the model for at least 50 epochs. You must reset the state of the RNN before starting each epoch.

Similar to task 2, the code is re-written to stop training if there is NaN in loss.

In [ ]:
# prepare data for training Stateful RNN
batch_size = 64
encoded_parts = np.array_split(encoded_data[:train_data_size], batch_size)
datasets = []

for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))

    datasets.append(dataset)

In [ ]:
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))

dataset = dataset.map(
    lambda X_batch, y_batch: (tf.one_hot(X_batch, depth=max_ids), y_batch)
)
dataset = dataset.prefetch(1)

In [ ]:
stateful_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2,
                     batch_input_shape=[batch_size, None, max_ids]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_ids,
                                                    activation='softmax'))
])

prefix = 'stateful_rnn'
save_model_and_halt_training = SaveModelAndHaltTraining(prefix)

In [ ]:
# callback to reset states
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [ ]:
stateful_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
steps_per_epoch = train_data_size // batch_size // n_steps

stateful_model.fit(dataset, steps_per_epoch=steps_per_epoch, epochs=50,
                   callbacks=[save_model_and_halt_training, ResetStatesCallback()])

Epoch 1/50
1094/1094 [==============================] - ETA: 0s - loss: 2.2782
Saving file: stateful_rnn_ep_1.h5
Epoch loss: 2.2781949043273926
1094/1094 [==============================] - 304s 274ms/step - loss: 2.2782
Epoch 2/50


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1094/1094 [==============================] - ETA: 0s - loss: 2.2559
Saving file: stateful_rnn_ep_2.h5
Epoch loss: 2.255906820297241
1094/1094 [==============================] - 300s 275ms/step - loss: 2.2559
Epoch 3/50
1094/1094 [==============================] - ETA: 0s - loss: 2.0347
Saving file: stateful_rnn_ep_3.h5
Epoch loss: 2.0347084999084473
1094/1094 [==============================] - 299s 274ms/step - loss: 2.0347
Epoch 4/50
1094/1094 [==============================] - ETA: 0s - loss: 1.9535
Saving file: stateful_rnn_ep_4.h5
Epoch loss: 1.953519344329834
1094/1094 [==============================] - 299s 274ms/step - loss: 1.9535
Epoch 5/50
1094/1094 [==============================] - ETA: 0s - loss: 1.8982
Saving file: stateful_rnn_ep_5.h5
Epoch loss: 1.8982489109039307
1094/1094 [==============================] - 301s 275ms/step - loss: 1.8982
Epoch 6/50
1094/1094 [==============================] - ETA: 0s - loss: 1.7924
Saving file: stateful_rnn_ep_6.h5
Epoch loss: 1.792440

In [20]:
model_path = "stateless_rnn_ep_50.h5"
loaded_model = keras.models.load_model(model_path)

In [21]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [22]:
model = loaded_model

In [23]:
X_new = preprocess(["How are yo"])
Y_pred = model.predict(X_new)
print(Y_pred)
Y_pred_classes = np.argmax(Y_pred, axis=-1)
predicted_text = tokenizer.sequences_to_texts(Y_pred_classes + 1)[0][-1]  # Last character of the first sentence

1/1 [==============================] - 1s 1s/step
[[[8.71104151e-02 3.73048723e-01 3.43304165e-02 1.32043451e-01
   1.89094439e-01 1.20781772e-01 2.43124962e-02 1.15181711e-02
   4.93453350e-03 1.02112454e-03 3.96172097e-03 4.42617340e-03
   4.52351244e-03 1.43095374e-03 8.88224924e-04 7.08747830e-04
   3.80122838e-05 2.64671189e-03 5.71538185e-05 1.39266439e-03
   6.91542809e-05 2.67091492e-12 7.34530913e-04 3.89263448e-12
   2.39440911e-18 1.61793869e-04 5.07803867e-04 4.35515467e-05
   7.49716908e-16 6.66290653e-05 7.63402945e-14 8.34543545e-12
   5.94975700e-06 5.61344896e-05 7.51288535e-05 1.13549009e-13
   1.11982333e-10 5.98973822e-12 2.10037664e-11 1.31328655e-08
   2.09435876e-12 7.86021096e-14 8.13488168e-06 5.24563461e-07
   1.00511672e-15 1.25177201e-06 1.49210835e-11 8.37273133e-12
   5.02569687e-12 4.43088517e-11 9.84529515e-13 2.17815106e-11
   9.53033694e-11 5.45328409e-15 7.84426568e-09 2.70986629e-13
   7.58134053e-33 3.62714437e-12 7.18899987e-24 6.02580458e-14
   1.

In [24]:
tf.random.set_seed(42)
tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()

array([[0, 1, 0, 2, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 2, 1, 0, 2, 1,
        0, 1, 2, 1, 1, 1, 2, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 2]])

## 4. Build a function “complete_text,” which will receive two parameters:
  - (i) a single character and
  - (ii) temperature – and will generate text of length 100 characters.

In [25]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [26]:
tf.random.set_seed(42)
next_char("How are yo", temperature=1)

1/1 [==============================] - 1s 604ms/step


'u'

In [27]:
def complete_text(text, n_chars=100, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

## 5. Run “complete_text,” which will call the stateless RNN model with three different characters: “a,” “s,” and “t” with three different temperature values of your choice and provide corresponding three different outputs.

In [29]:
tf.random.set_seed(42)
print(complete_text("a", temperature=0.6)) # "a" is the first character to start with.

1/1 [==============================] - 0s 32ms/step
anding is the funner the film of the screent . 
the prever than anything that the paracter the film o


In [30]:
tf.random.set_seed(42)
print(complete_text("s", temperature=0.4))

1/1 [==============================] - 0s 32ms/step
s any movie is his movies and a between and hoolling is the film . 
out one for the screen the film o


In [31]:
tf.random.set_seed(42)
print(complete_text("t", temperature=0.6)) # "t" is the first character to start with.

1/1 [==============================] - 0s 34ms/step
t him with a schole films is the show movies . 
the prever than anything that the paracter the film o


## 6. Repeat step #5 for the stateful RNN model.

In [32]:
model_path = "stateful_rnn_ep_18.h5"
loaded_model = keras.models.load_model(model_path)

In [33]:
model = loaded_model

In [35]:
tf.random.set_seed(42)
print(complete_text("a", temperature=0.2))

1/1 [==============================] - 0s 175ms/step
ak , and the story of the characters and screenplay . 
it is a strong and story because the film , an


In [36]:
tf.random.set_seed(42)
print(complete_text("s", temperature=0.4))

1/1 [==============================] - 0s 107ms/step
s all the tries . 
and the movie is a complete of the surprise in his did for the film . 
the things 


In [34]:
tf.random.set_seed(42)
print(complete_text("t", temperature=0.6))

1/1 [==============================] - 0s 99ms/step
ter . 
a men and delighted in the movie who must really the film . 
where should do the movie that ma
